## Magics aren't magic

Magics are cool, magics can be complicated, but magics aren't magic. They are python code that jupyter let's you shortcut with a `%` or `%%` symbol. The major difference between a magic and an alias, is that the magic gets special access to the notebook state. 

In [ ]:
from IPython.core.magic import register_line_magic, register_cell_magic, needs_local_scope
from IPython.core.magic import register_line_cell_magic

Source code for individual magics is stored in 
[this section of the IPython Github](https://github.com/ipython/ipython/blob/master/IPython/core/magics)

## The simplest custom line magic

### NOTE: Get better example and point out the line doesnt need to be valid

`@register_line_magic` is a python decorator that tells jupyter to make `funcname` available as `%funcname`, or if the decorator is passed a custom name, then %customname will call the magic.

First we make a normal function that takes a string and reverses the word order. 

In [ ]:
def revline(line):
    return " ".join(line.split()[::-1])

In [ ]:
revline("x = 4b + 3")

'3 + 4b = x'

In [ ]:
##Oops not a magic yet, let's rewrite it with the decorator above
%revline

UsageError: Line magic function `%revline` not found.


In [ ]:
@register_line_magic
def revline(line):
    return " ".join(line.split()[::-1])

In [ ]:
#underlying function still works
revline("x = 4b + 3")

'3 + 4b = x'

In [ ]:
# using the magic we don't need quotes anymore, if we put them in theyll be considered part of the line
%revline x = 4b + 3

'3 + 4b = x'

In [ ]:
#name it something different than the function
@register_line_magic("rev")
def revline(line):
    return " ".join(line.split()[::-1])

In [ ]:
%rev x = 4b + 3

'3 + 4b = x'

That's it. Congratulations, you've made your own magic, but right now it's no better than an alias

## Getting access to notebook state

Use the `@needs_local_scope` decorator to get access to the notebook namespace. It does this by passing in a new argument, `local_ns` to our function, so we will need to start accepting that. Let's write a new magic to accept `local_ns`, and then print out some info so we can see what we're getting

Note that the order of the decorators does matter, and registering the magic should always be the outermost part.

In [ ]:
@register_line_magic("mag")
@needs_local_scope
def some_magic(line, local_ns):
    print("Namespace is stored as a", type(local_ns))
    keys = [k for k in local_ns.keys() if not k.startswith('_')]
    print(f"There are {len(keys)} keys you can access")
    print("They are...\n", keys)

In [ ]:
# We still have to pass in a line, but we aren't using it so let's use empty quotes
%mag ""

Namespace is stored as a <class 'dict'>
There are 11 keys you can access
They are...
 ['In', 'Out', 'get_ipython', 'exit', 'quit', 'revline', 'register_line_magic', 'register_cell_magic', 'needs_local_scope', 'register_line_cell_magic', 'some_magic']


### The most useful pieces of notebook state

#### get_ipython() - Returns a reference to the current InteractiveShell

This will give you access to the Notebook object and let you interact with it in really cool ways. This is the part that can make magics seem like magic

In [ ]:
ip = get_ipython()

In [ ]:
ip.set_next_input("A cell magically appears")

In [ ]:
A cell magically appears

In [ ]:
ip.ask_yes_no("Was that cool or what?")

Was that cool or what? yes


True

#### `In` - A list of commands so far

In [ ]:
In

['',
 'def revline(line):\n    return " ".join(line.split()[::-1])',
 'revline("x = 4b + 3")',
 "##Oops not a magic yet, let's rewrite it with the decorator above\nget_ipython().run_line_magic('revline', '')",
 '@register_line_magic\ndef revline(line):\n    return " ".join(line.split()[::-1])',
 'from IPython.core.magic import register_line_magic, register_cell_magic, needs_local_scope\nfrom IPython.core.magic import register_line_cell_magic',
 'def revline(line):\n    return " ".join(line.split()[::-1])',
 'revline("x = 4b + 3")',
 "##Oops not a magic yet, let's rewrite it with the decorator above\nget_ipython().run_line_magic('revline', '')",
 '@register_line_magic\ndef revline(line):\n    return " ".join(line.split()[::-1])',
 '#underlying function still works\nrevline("x = 4b + 3")',
 "# using the magic we don't need quotes anymore, if we put them in theyll be considered part of the line\nget_ipython().run_line_magic('revline', 'x = 4b + 3')",
 '#name it something different than th

Access like a normal list, `In[0]` is the first command that was issued this session, `In[4]` is the fifth, `In[-1]` is the most recent

#### `Out` - A dict containing command indexes and their outputs

Out is a dictionary where each key is the index of a command that has been run during the session, and values are the output of that command. This is confusing so let's unpack it a bit.

![Explaining the Out dict](images/magicOut.png)

In [ ]:
Out

{2: '3 + 4b = x',
 7: '3 + 4b = x',
 10: '3 + 4b = x',
 11: '3 + 4b = x',
 13: '3 + 4b = x',
 18: True,
 19: ['',
  'def revline(line):\n    return " ".join(line.split()[::-1])',
  'revline("x = 4b + 3")',
  "##Oops not a magic yet, let's rewrite it with the decorator above\nget_ipython().run_line_magic('revline', '')",
  '@register_line_magic\ndef revline(line):\n    return " ".join(line.split()[::-1])',
  'from IPython.core.magic import register_line_magic, register_cell_magic, needs_local_scope\nfrom IPython.core.magic import register_line_cell_magic',
  'def revline(line):\n    return " ".join(line.split()[::-1])',
  'revline("x = 4b + 3")',
  "##Oops not a magic yet, let's rewrite it with the decorator above\nget_ipython().run_line_magic('revline', '')",
  '@register_line_magic\ndef revline(line):\n    return " ".join(line.split()[::-1])',
  '#underlying function still works\nrevline("x = 4b + 3")',
  "# using the magic we don't need quotes anymore, if we put them in theyll be con

## A real magic in 7 lines: writing `%recall` from scratch

Let's quickly reinvent the %recall magic, that we saw in the "might be useful" section. 

Before we do, I want to bring your attention to how magic this magic seemed before. You type `%recall 22` and it pops into existence a cell with the 22nd command you issued this session. 

Now let's see it from a new perspective, as pretty straightforward Python, now that we know how to access `In` and the notebook object via `get_ipython()`

In [ ]:
@register_line_magic("rec")
@needs_local_scope
def recall_magic(num, local_ns):
    commands = local_ns["In"]
    command_to_recall = commands[int(num)]
    ip = get_ipython()
    ip.set_next_input(command_to_recall)

In [ ]:
%rec 10

In [ ]:
#underlying function still works
revline("x = 4b + 3")

If you want to be a stickler, `%recall` also allows you to pass no arg, and returns a list of all commands from the session. This is an easy fix by changing...  

`command_to_recall = commands[int(num)]`  

to  

`command_to_recall = str(commands) if not num else commands[int(num)]` 

but I omitted it to simplify the code

## Our First Cell Magic

To create a cell magic, just change `@register_line_magic` to be `@register_cell_magic`, and alter your function to take two arguments, `line` and `cell`. This naming is slightly confusing as this is only a cell magic, not a line and cell magic (which we'll see next), so `line` is really just the arguments to the magic

In [ ]:
@register_cell_magic
def cell_magic(line, cell):
    print(f"Line is of type {type(line)} and contains '{line}'")
    print(f"Cell is of type {type(cell)} and contains '{cell}'")

In [ ]:
%%cell_magic argument1 argument2
Line 1
Line 2

Line is of type <class 'str'> and contains 'argument1 argument2'
Cell is of type <class 'str'> and contains 'Line 1
Line 2
'


### Cell magic from scratch: Building a line, word, and char counter

Here's a not all that useful example cell magic, but it does something we couldn't do before, counts the contents of the cell

In [ ]:
@register_cell_magic("countit")
def cell_counter(line, cell):
    lns = cell.strip().split('\n')
    nlines = len(lns)
    nwords = sum([len(ln.strip().split(' ')) for ln in lns])
    nchars = len(cell)
    print(f"Lines: {nlines}\nWords: {nwords}\nChars: {nchars}")

In [ ]:
%%countit
for i in range(36):
    print(i)

Lines: 2
Words: 5
Chars: 33


In [ ]:
%%countit
this is not code!
    How in the world is this not raising an error in a code cell?!?!?

Lines: 2
Words: 18
Chars: 88


## Line and Cell Magic at the same time: EXAMPLE NEEDED

In [ ]:
@register_line_cell_magic
def lcmagic(line, cell=None):
    if cell is None:
        print("Called as line magic")
        return line
    else:
        print("Called as cell magic")
        return line, cell

## Storing Magic State between calls by creating a class 

You can also create line, cell, and line+cell magics by inheriting from the `Magics` class, and putting in a few decorators. There are two cases that this appears to be better than what we did above

1. You want to have some type of state.
2. You are creating multiple magics that logically go together. This is how the [IPython Magics Github](https://github.com/ipython/ipython/tree/master/IPython/core/magics) is written. Logging magics are in logging.py in a class called `LoggingMagics`, timing/profiling are in execution.py in the `ExecutionMagics` class

In [ ]:
from IPython.core import magic_arguments
from IPython.core.magic import line_magic, cell_magic, line_cell_magic, Magics, magics_class
 
@magics_class
class TestMagics(Magics):
    @cell_magic
    def hello(self, line='', cell=None):
        print('hello ' + cell)
 
    @line_magic
    def hi(self, line):
        print(self.shell)
        print('hi ' + line)
 
ip = get_ipython()
ip.register_magics(TestMagics)

In [ ]:
%hi jupyter

hi jupyter



### Best way to register magics to be always available

https://stackoverflow.com/a/34655190/5042053